## Comparison of Model 2 created with basic RFM model

In [54]:
# Import The Libraries
import pandas as pd, matplotlib.pyplot as plt, numpy as np, seaborn as sns, scipy as sp, warnings
warnings.filterwarnings("ignore")

In [28]:
#read prod category and rfm data
prod_cat = pd.read_pickle('PCA_RED_DESC.pkl')
rfm = pd.read_pickle('rfm.pkl')
final_data = pd.merge(prod_cat,rfm,on='Customer ID')
final_data_new = pd.merge(prod_cat,rfm,on='Customer ID')
final_data.drop(['RFM_Segment','Customer ID','R','F','M','Final_Price','RFM_Score'],axis = 'columns',inplace=True)
final_data.rename(columns={0:'0',1:'1',2:'2',3:'3',4:'4',5:'5'})
#Rescaling
from sklearn.preprocessing import MinMaxScaler
numeric_columns = final_data.columns.values.tolist()
scaler = MinMaxScaler() 
final_data[numeric_columns] = scaler.fit_transform(final_data[numeric_columns])
from sklearn.cluster import KMeans
scores_2 = []

range_values = range(1, 20)

for i in range_values:
    
    kmeans = KMeans(n_clusters= i)
    kmeans.fit(final_data)
    scores_2.append(kmeans.inertia_)
kmeans = KMeans(3)
kmeans.fit(final_data)
labels = kmeans.labels_
y_kmeans = kmeans.fit_predict(final_data)
df_cluster_dr = pd.concat([final_data_new, pd.DataFrame({'cluster_final':labels})], axis = 1)
print(df_cluster_dr['cluster_final'].value_counts())

0    3601
1    1499
2     778
Name: cluster_final, dtype: int64


In [16]:
dataset=pd.read_excel('C:/Personal/NUIG/Project-DA/online_retail_II.xlsx',sheet_name=None)
data_before = dataset['Year 2009-2010']
data_after = dataset['Year 2010-2011']

In [73]:
#select one customer in both datasets
#get the cluster of that customer
#recommend top 20 customer wala thing
#remove all descriptions he already bought
#check in after dataset how many matches

In [17]:
#remove all customers with NA customer ID
data_before = data_before[data_before['Customer ID'].notna()]
data_after = data_after[data_after['Customer ID'].notna()]
cust_before = (data_before.sample(n=20)['Customer ID']).tolist()

In [31]:
percent_match = []
for cust in cust_before:
    if cust in data_after['Customer ID']:
        cluster = df_cluster_dr[df_cluster_dr['Customer ID'] == cust]
        if cluster['cluster_final'].item() == 0:
            d = kmeans.transform(final_data)[:, 0]
        elif cluster['cluster_final'].item() == 1:
            d = kmeans.transform(final_data)[:, 1]
        else:
            d = kmeans.transform(final_data)[:, 2]
        #select 20 nearest points to that cluster and get their indices
        ind = np.argsort(d)[::][:30]
        #get the data of these 20 customers
        data_recom_id = df_cluster_dr[df_cluster_dr.index.isin(ind)]
        #get the desc of that customer before recommending
        data_bf = data_before[data_before['Customer ID'] == cust]['Description'].apply(str).unique()
        #results of recommendation
        data_recom = data_recom_id.merge(data_before, on='Customer ID', how='left')
        descriptions=data_recom['Description'].apply(str).unique()
        des_final = np.setdiff1d(descriptions,data_bf)
        #evaluate with data_after
        data_input = df_cluster_dr[df_cluster_dr['Customer ID'] == cust]
        data_after_1 = data_input.merge(data_after, on='Customer ID', how='left')
        descriptions_after=data_after_1['Description'].apply(str).unique()
        percent = len([x for x in descriptions_after if x in des_final])/len(descriptions_after)
        print('Description matching rate',round(percent*100,2),cluster['cluster_final'].item(),data_recom_id['cluster_final'].unique())
        percent_match.append(percent)
    else:
        print('Customer Not present')

Customer Not present
Customer Not present
Description matching rate 38.84 0 [0]
Description matching rate 32.05 0 [0]
Description matching rate 10.43 0 [0]
Description matching rate 20.16 0 [0]
Customer Not present
Description matching rate 28.44 0 [0]
Description matching rate 14.29 0 [0]
Description matching rate 12.76 0 [0]
Description matching rate 36.36 0 [0]
Description matching rate 100.0 1 [1]
Description matching rate 11.54 1 [1]
Customer Not present
Description matching rate 32.2 0 [0]
Customer Not present
Description matching rate 100.0 1 [1]
Description matching rate 23.74 0 [0]
Description matching rate 31.51 0 [0]
Customer Not present


In [32]:
sum(percent_match)/len(percent_match)

0.35164894646270106

In [38]:
#read prod category and rfm data
prod_cat = pd.read_pickle('PCA_RED_DESC.pkl')
rfm = pd.read_pickle('rfm.pkl')
final_data_1 = pd.merge(prod_cat,rfm,on='Customer ID')
final_data_1_new = pd.merge(prod_cat,rfm,on='Customer ID')
final_data_1.rename(columns={0:'0',1:'1',2:'2',3:'3',4:'4',5:'5'})
final_data_1.drop(['RFM_Segment','Customer ID','R','F','M','Final_Price',0,1,2,3,4,5],axis = 'columns',inplace=True)
#Rescaling
from sklearn.preprocessing import MinMaxScaler
numeric_columns = final_data_1.columns.values.tolist()
scaler = MinMaxScaler() 
final_data_1[numeric_columns] = scaler.fit_transform(final_data_1[numeric_columns])
from sklearn.cluster import KMeans
scores_2 = []

range_values = range(1, 20)

for i in range_values:
    
    kmeans_1 = KMeans(n_clusters= i)
    kmeans_1.fit(final_data_1)
    scores_2.append(kmeans_1.inertia_)
kmeans_1 = KMeans(3)
kmeans_1.fit(final_data_1)
labels_1 = kmeans_1.labels_
y_kmeans_1 = kmeans_1.fit_predict(final_data_1)
df_cluster_dr_1 = pd.concat([final_data_1_new, pd.DataFrame({'cluster_final':labels_1})], axis = 1)
print(df_cluster_dr_1['cluster_final'].value_counts())

2    2129
0    1985
1    1764
Name: cluster_final, dtype: int64


In [39]:
from sklearn.metrics import silhouette_samples,silhouette_score
for i in set(df_cluster_dr_1['cluster_final']):
    customer_count = len(df_cluster_dr_1[df_cluster_dr_1['cluster_final']==i])
    r_mean = round(np.mean(df_cluster_dr_1[df_cluster_dr_1['cluster_final']==i]['Recency']),2)
    f_mean = round(np.mean(df_cluster_dr_1[df_cluster_dr_1['cluster_final']==i]['Frequency']),2)
    m_mean = round(np.mean(df_cluster_dr_1[df_cluster_dr_1['cluster_final']==i]['Monetary_value']),2)
    print('Count:',i,customer_count,r_mean,f_mean,m_mean)
    
silhouette_avg = silhouette_score(final_data, y_kmeans)
print('score de silhouette: {:<.3f}'.format(silhouette_avg))

Count: 0 1985 120.28 52.75 274.99
Count: 1 1764 483.18 37.28 317.88
Count: 2 2129 40.61 286.03 543.67
score de silhouette: 0.680


In [52]:
percent_match = []
for cust in cust_before:
    if cust in data_after['Customer ID']:
        cluster = df_cluster_dr_1[df_cluster_dr_1['Customer ID'] == cust]
        if cluster['cluster_final'].item() == 2:
            d = kmeans_1.transform(final_data_1)[:, 0]
        elif cluster['cluster_final'].item() == 0:
            d = kmeans_1.transform(final_data_1)[:, 1]
        else:
            d = kmeans_1.transform(final_data_1)[:, 2]
        #select 20 nearest points to that cluster and get their indices
        ind = np.argsort(d)[::][:30]
        #get the data of these 20 customers
        data_recom_id = df_cluster_dr_1[df_cluster_dr_1.index.isin(ind)]
        #get the desc of that customer before recommending
        data_bf = data_before[data_before['Customer ID'] == cust]['Description'].apply(str).unique()
        #results of recommendation
        data_recom = data_recom_id.merge(data_before, on='Customer ID', how='left')
        descriptions=data_recom['Description'].apply(str).unique()
        des_final = np.setdiff1d(descriptions,data_bf)
        #evaluate with data_after
        data_input = df_cluster_dr_1[df_cluster_dr_1['Customer ID'] == cust]
        data_after_1 = data_input.merge(data_after, on='Customer ID', how='left')
        descriptions_after=data_after_1['Description'].apply(str).unique()
        percent = len([x for x in descriptions_after if x in des_final])/len(descriptions_after)
        print('Description matching rate',round(percent*100,2),cluster['cluster_final'].item(),data_recom_id['cluster_final'].unique())
        percent_match.append(percent)
    else:
        print('Customer Not present')

Customer Not present
Customer Not present
Description matching rate 34.71 2 [2]
Description matching rate 30.77 2 [2]
Description matching rate 11.23 2 [2]
Description matching rate 17.83 2 [2]
Customer Not present
Description matching rate 29.38 2 [2]
Description matching rate 11.43 2 [2]
Description matching rate 14.11 2 [2]
Description matching rate 38.38 2 [2]
Description matching rate 0.0 1 [1]
Description matching rate 19.23 0 [0]
Customer Not present
Description matching rate 39.83 2 [2]
Customer Not present
Description matching rate 0.0 1 [1]
Description matching rate 26.52 2 [2]
Description matching rate 31.51 2 [2]
Customer Not present


In [53]:
sum(percent_match)/len(percent_match)

0.21780571649378147

In [55]:
#read prod category and rfm data
prod_cat = pd.read_pickle('PCA_RED_DESC.pkl')
rfm = pd.read_pickle('rfm.pkl')
final_data_1 = pd.merge(prod_cat,rfm,on='Customer ID')
final_data_1_new = pd.merge(prod_cat,rfm,on='Customer ID')
final_data_1.rename(columns={0:'0',1:'1',2:'2',3:'3',4:'4',5:'5'})
final_data_1.drop(['RFM_Segment','Customer ID','R','F','M','Final_Price',0,1,2,3,4,5,'RFM_Score'],axis = 'columns',inplace=True)
#Rescaling
from sklearn.preprocessing import MinMaxScaler
numeric_columns = final_data_1.columns.values.tolist()
scaler = MinMaxScaler() 
final_data_1[numeric_columns] = scaler.fit_transform(final_data_1[numeric_columns])
from sklearn.cluster import KMeans
scores_2 = []

range_values = range(1, 20)

for i in range_values:
    
    kmeans_1 = KMeans(n_clusters= i)
    kmeans_1.fit(final_data_1)
    scores_2.append(kmeans_1.inertia_)
kmeans_1 = KMeans(3)
kmeans_1.fit(final_data_1)
labels_1 = kmeans_1.labels_
y_kmeans_1 = kmeans_1.fit_predict(final_data_1)
df_cluster_dr_1 = pd.concat([final_data_1_new, pd.DataFrame({'cluster_final':labels_1})], axis = 1)
print(df_cluster_dr_1['cluster_final'].value_counts())

1    3602
2    1498
0     778
Name: cluster_final, dtype: int64


In [56]:
from sklearn.metrics import silhouette_samples,silhouette_score
for i in set(df_cluster_dr_1['cluster_final']):
    customer_count = len(df_cluster_dr_1[df_cluster_dr_1['cluster_final']==i])
    r_mean = round(np.mean(df_cluster_dr_1[df_cluster_dr_1['cluster_final']==i]['Recency']),2)
    f_mean = round(np.mean(df_cluster_dr_1[df_cluster_dr_1['cluster_final']==i]['Frequency']),2)
    m_mean = round(np.mean(df_cluster_dr_1[df_cluster_dr_1['cluster_final']==i]['Monetary_value']),2)
    print('Count:',i,customer_count,r_mean,f_mean,m_mean)
    
silhouette_avg = silhouette_score(final_data, y_kmeans)
print('score de silhouette: {:<.3f}'.format(silhouette_avg))

Count: 0 778 598.14 29.73 346.31
Count: 1 3602 52.77 186.33 408.92
Count: 2 1498 348.54 56.84 348.29
score de silhouette: 0.680


In [57]:
percent_match = []
for cust in cust_before:
    if cust in data_after['Customer ID']:
        cluster = df_cluster_dr_1[df_cluster_dr_1['Customer ID'] == cust]
        if cluster['cluster_final'].item() == 2:
            d = kmeans_1.transform(final_data_1)[:, 0]
        elif cluster['cluster_final'].item() == 0:
            d = kmeans_1.transform(final_data_1)[:, 1]
        else:
            d = kmeans_1.transform(final_data_1)[:, 2]
        #select 20 nearest points to that cluster and get their indices
        ind = np.argsort(d)[::][:30]
        #get the data of these 20 customers
        data_recom_id = df_cluster_dr_1[df_cluster_dr_1.index.isin(ind)]
        #get the desc of that customer before recommending
        data_bf = data_before[data_before['Customer ID'] == cust]['Description'].apply(str).unique()
        #results of recommendation
        data_recom = data_recom_id.merge(data_before, on='Customer ID', how='left')
        descriptions=data_recom['Description'].apply(str).unique()
        des_final = np.setdiff1d(descriptions,data_bf)
        #evaluate with data_after
        data_input = df_cluster_dr_1[df_cluster_dr_1['Customer ID'] == cust]
        data_after_1 = data_input.merge(data_after, on='Customer ID', how='left')
        descriptions_after=data_after_1['Description'].apply(str).unique()
        percent = len([x for x in descriptions_after if x in des_final])/len(descriptions_after)
        print('Description matching rate',round(percent*100,2),cluster['cluster_final'].item(),data_recom_id['cluster_final'].unique())
        percent_match.append(percent)
    else:
        print('Customer Not present')

Customer Not present
Customer Not present
Description matching rate 15.7 1 [2]
Description matching rate 12.82 1 [2]
Description matching rate 5.61 1 [2]
Description matching rate 7.75 1 [2]
Customer Not present
Description matching rate 17.06 1 [2]
Description matching rate 8.57 1 [2]
Description matching rate 5.17 1 [2]
Description matching rate 17.17 1 [2]
Description matching rate 100.0 2 [1]
Description matching rate 50.0 2 [1]
Customer Not present
Description matching rate 22.03 1 [2]
Customer Not present
Description matching rate 100.0 2 [1]
Description matching rate 11.62 1 [2]
Description matching rate 14.61 1 [2]
Customer Not present


In [58]:
sum(percent_match)/len(percent_match)

0.27723431230187